# Check on the files exported by Equivital Qiosk
for RITMO

TODO

Plot and review measurements from equivital sensors as output by the Equivital Manager app.
(Every program produces different formated files and it's a pain.)

In [2]:
import sys
import os
import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import heartpy as hp

from scipy.signal import butter,filtfilt
from scipy import interpolate
from scipy.interpolate import interp1d

## Definitions for data extraction

In [175]:
def min_dets(eq_file_loc): # for csv files output by the qiosk app
    filings = eq_file_loc.split('\\')
    file_name = filings[-1]
    f = file_name.split('-')
    Signal = f[0]
    DevName = f[1]#filings[-2]
    DevID = int(f[2])
    fileDate = int(f[3][:6]) # interpret as datetime datatype?
    # sometimes the session numbering fails and we get files with the same session number but an additiona _0 or _1
    # how to number this? What errors are producing these session numbers?
    if len(f[3].split('_'))==2: # we have an additional numbering to work into the sessions. :[
        Sessn1 = int(f[3].split('_')[0][6:8])
        Sessn2 = int(f[3].split('_')[1].split('.')[0])
        Session = (Sessn1+1)*100 + Sessn2+1 # yes this makes the session numbers huge out of nowhere, but it won't overlap with QIOSKs proper numbering that goes up to 99
    else:
        Session = int(f[3][6:8])
    fileSize = os.path.getsize(eq_file_loc)

    File_dets={'Signal':Signal, #f[-2].split('_')[-1],
       'DevName':DevName,
       'ID':DevID, 
       'Date':fileDate,
       'Session':Session,
       'FileName':file_name,
       'FileType':'csv',
       'FileSize': fileSize,
       'FullLoc':eq_file_loc}
    return File_dets

def data_dets(eq_file_loc): #rec_start = V['DateTime'].iloc[0]
    # this file pulls recording details from the file name and from inside file to aggregate all metadata
    filings = eq_file_loc.split('\\')
    file_name = filings[-1]
    f = file_name.split('-')
    Signal = f[0]
    DevName = f[1]#filings[-2]
    DevID = int(f[2])
    fileDate = int(f[3][:6]) # interpret as datetime datatype?
    # sometimes the session numbering fails and we get files with the same session number but an additiona _0 or _1
    # how to number this? What errors are producing these session numbers?
    if len(f[3].split('_'))==2: # we have an additional numbering to work into the sessions. :[
        Sessn1 = int(f[3].split('_')[0][6:8])
        Sessn2 = int(f[3].split('_')[1].split('.')[0])
        Session = (Sessn1+1)*100 + Sessn2+1 # yes this makes the session numbers huge out of nowhere, but it won't overlap with QIOSKs proper numbering that goes up to 99
    else:
        Session = int(f[3][6:8])
    fileSize = os.path.getsize(eq_file_loc)
    
    V = pd.read_csv(eq_file_loc,skipinitialspace=True)
    if len(V)==0:
        File_dets={'Signal':Signal, #f[-2].split('_')[-1],
           'DevName':DevName,
           'ID':DevID, 
           'Date':fileDate,
           'Session':Session,
           'FileName':file_name,
           'FileType':'csv',
           'FileSize': fileSize,
           'RecStart':pd.to_datetime('2020-02-02 02:02:00.00+0000'), # fake start
           'FullLoc':eq_file_loc}
        return File_dets
    
    else:
        V['DateTime'] = pd.to_datetime(V['DateTime'])
        rec_start = V['DateTime'].iloc[0]
        rec_end = V['DateTime'].iloc[-1]
        rec_dur=(rec_end-rec_start).total_seconds()
        Batt_start = V['BATTERY(mV)'].iloc[0]
        Batt_end = V['BATTERY(mV)'].iloc[-1]
        Batt_spend=(Batt_end-Batt_start)     
        
        a = V.loc[:,['SENSOR ID', 'SUBJECT ID', 'SUBJECT AGE', 'HR(BPM)',
           'HRC(%)', 'BELT OFF', 'LEAD OFF', 'MOTION', 'BODY POSITION']].mode().loc[0]
        DevNames = V.loc[:,'SUBJECT ID'].unique()

        File_dets={'Signal':Signal, #f[-2].split('_')[-1],
           'DevName':DevName,
           'ID':DevID, 
           'Date':fileDate,
           'Session':Session,
           'FileName':file_name,
           'FileType':'csv',
           'FileSize': fileSize,
           'RecStart':rec_start,
           'RecEnd':rec_end,
           'Duration':rec_dur,
           'BatteryStart':Batt_start,
           'BatteryEnd':Batt_end,
           'BatteryChange(mV)':Batt_spend,
           'FullLoc':eq_file_loc,
           'SubjectNames': DevNames}
        File_dets.update(a) # dic0.update(dic1)
        return File_dets

def test_plot_signals(V): # V is a qiosk file read into pandas
    if len(V)>2:
        V['DateTime'] = pd.to_datetime(V['DateTime'])
        W = V.select_dtypes(include=['int64','float64'])
        W.set_index(V['DateTime'],inplace=True)
        cols = W.columns
        # excerpt a minute of signal from the middle of the recording
        if V['DateTime'].iloc[-1]-V['DateTime'].iloc[0]>pd.to_timedelta(120,'s'):
            t1 =  V['DateTime'].iloc[int(len(V)/2)]
            t2 = t1+pd.to_timedelta(60,'s')
            X = W.loc[W.index>t1,:].copy()
            X = X.loc[X.index<t2,:].copy()
            for c in cols:
                fig, (ax1, ax2) = plt.subplots(1,2,figsize=[15,2])
                W[c].plot(ax=ax1)
                ax1.set_ylabel(c)
                X.loc[:,c].plot(ax=ax2)
                ax2.set_xlabel('60 seconds')
                plt.show()
        else:
            for c in cols:
                fig, (ax1) = plt.subplots(1,1,figsize=[15,2])
                W[c].plot(ax=ax1)
                ax1.set_ylabel(c)
                plt.show()
    else:
        print('Not enough data')
        
def test_plot_signals_interval(V,t1,t2): # V is a qiosk file read into pandas
    # its on you to be sure these time stamps are within the recording interval of the file
    if len(V)>2:
        V['DateTime'] = pd.to_datetime(V['DateTime'])
        W = V.select_dtypes(include=['int64','float64'])
        W.set_index(V['DateTime'],inplace=True)
        cols = W.columns
        X = W.loc[W.index>t1,:].copy()
        X = X.loc[X.index<t2,:].copy()
        for c in cols:
            fig, (ax1) = plt.subplots(1,1,figsize=[15,2])
            X.loc[:,c].plot(ax=ax1)
            ax1.set_ylabel(c)
            plt.show()
    else:
        print('No data')


In [191]:
        
def matched_files(eq_file_loc,data_path):
    # from the location of a good file and the location of other files, retrieve the location of all matching files
    dfile = min_dets(eq_file_loc)
    
    # retrieve the files in that path that match 
    file_locs = []
    for root, dirs, files in os.walk(data_path):
        for file in files:
            if(file.lower().endswith(".csv")):
                file_locs.append(os.path.join(root,file))
    k=[]
    for file in file_locs:
        if not file.lower().endswith('recordings.csv'):
            File_dets=min_dets_sem(file)
            k.append(File_dets)
    df_files=pd.DataFrame(data=k)

    match_fields = ['ID','DevName','Date','Session']

    matched_files = df_files.loc[df_files['ID'] == dfile['ID']]
    for mf in match_fields[1:]:
        matched_files = matched_files.loc[matched_files[mf] == dfile[mf]]

    return list(matched_files['FullLoc'])+list(matched_files['SEMLoc'].unique())
    

def min_dets_sem(eq_file_loc): # for files output by the lab manager desktop app, so far
    w = eq_file_loc.split('\\')
    file_name = w[-1]
    f = file_name.split('-')
    Signal = f[0]
    DevName = f[1]#filings[-2]
    DevID = int(f[2])
    fileDate = int(f[3][:6]) # interpret as datetime datatype?
    # sometimes the session numbering fails and we get files with the same session number but an additiona _0 or _1
    # how to number this? What errors are producing these session numbers?
    if len(f[3].split('_'))==2: # we have an additional numbering to work into the sessions. :[
        Sessn1 = int(f[3].split('_')[0][6:8])
        Sessn2 = int(f[3].split('_')[1].split('.')[0])
        Session = (Sessn1+1)*100 + Sessn2+1 # yes this makes the session numbers huge out of nowhere, but it won't overlap with QIOSKs proper numbering that goes up to 99
    else:
        Session = int(f[3][6:8])
    fileSize = os.path.getsize(eq_file_loc)
    
    # assuming qiosks file structure is consistent, SEM file naming should be reliable
    if eq_file_loc.startswith('C:\\Users\\Public\\Documents\\Equivital\\Equivital Manager Wizard\\'): # initial qiosk exports
        fn = w[-1].split('-')[-1][:-3]+'SEM'
        sem_path = w[:-3]+['Raw SEM Data',w[-2],fn]
        sem_loc = '\\'.join(sem_path)
    else:
        if w[-2].lower().startswith('csv'): # Use earlier details to formulate location and structure
            fn = file_name.split('-')[-1][:-3]+'SEM'
            sem_path = eq_file_loc.split('\\')[:-2] + ['SEM',DevName,fn]
            sem_loc = '\\'.join(sem_path)
        else:
            return []

    File_dets={'Signal':Signal, #f[-2].split('_')[-1],
       'DevName':DevName,
       'ID':DevID, 
       'Date':fileDate,
       'Session':Session,
       'FileName':file_name,
       'FileType':'csv',
       'FileSize': fileSize,
       'FullLoc':eq_file_loc,
       'SEMLoc': sem_loc}
    return File_dets

def qiosk_recordings(projectpath,projecttag):
    file_locs = []
    for root, dirs, files in os.walk(projectpath):
        for file in files:
            if(file.lower().endswith(".csv")):
                if file.lower().startswith('data'):
                    file_locs.append(os.path.join(root,file))
    if len(file_locs)>0:
        k=[]           
        for f in file_locs:
            File_dets=data_dets(f)
            if File_dets:
                k.append(File_dets)
        df_datafiles=pd.DataFrame(data=k)#
        df_datafiles=df_datafiles.sort_values(by='RecStart').reset_index(drop=True)
        df_datafiles.to_csv(projectpath + projecttag + '_Qiosk_recordings.csv')
        return df_datafiles
    else:
        print('Path is empty of DATA files.')
        return []

# Scan for DATA files

All recordinds exported by Qiosk produce four CSV files and one SEM file. One of the CSVs is a DATA file that contains metadata and some essential signal quality checks on 15 s intervals.

We can ID instances of signal recordings from statistics taken out of the DATA file, so that gets extracted first

In [192]:
# location of the files when exported by QIOSK
path = "C:\\Users\\Public\\Documents\\Equivital\\Equivital Manager Wizard\\"#Extracted SEM Data\\" #"../Equivital/Equivital Manager Wizard/Extracted SEM Data/"
#os.listdir(path)

df_datafiles = qiosk_recordings(path,'Present')
df_datafiles

Path is empty of DATA files.


[]

In [186]:
print('Number of DATA files: '+ str(len(df_datafiles)))
# key categories to structure of Qiosk output DATA files
print(df_datafiles['ID'].unique())
print(df_datafiles['DevName'].unique())
print(df_datafiles['Date'].unique())

Number of DATA files: 211
[3420415 5022164 5022060 5022138 5022126 5022139 5022141 5022050 5022281
 5022299 3420427 3420456 5022166 5022285 5022027 5022241 5022110 5022247
 5022198 5022193 5022222 5022058 3420755 3420426 5022077 5022156 5022297
 5022143 5022202 5022173 5022098 5022176 5022076 5022010 5022155 5022100
 5022161 5023926 5022154 5022238 5022244 5022149 5022127 5022031 5022062
 5022204 5022104 5022179 5022057 5022203 5022151 5022245 5022234 5022026
 5022184 5022051 5022009 5022006 5022117 5022073 5022152 5022256 5022101
 5022142 5022207 5022131 5022163 5022002 5022240 5022292]
['Bluetooth_Tester_1' 'Pilot_31' 'Pilot_32' 'Pilot_4' 'Pilot_5' 'Pilot_6'
 'Pilot_30' 'Pilot_7' 'Pilot_8' 'Pilot_9' 'Pilot_3' 'Pilot_2' 'Pilot_20'
 'Pilot_21' 'Pilot_22' 'Pilot_23' 'Pilot_24' 'Pilot_25' 'Pilot_26'
 'Pilot_27' 'Pilot_28' 'Pilot_29' 'Test1' 'Test3_On' 'Test4' 'Test5'
 'Test2' 'Test2_On' 'Pilot 39' 'Pilot 40' 'Pilot 41' 'Pilot 42' 'Pilot 43'
 'Pilot 38' 'Pilot 37' 'Bluetooth_Tester_2' 'Bl

## Define set of recordings to move
First specify a subset by either device number (singular) or day of recording event.

Next this subset can be refined by restricting recordings to a specific time of day: to have started before a particular time (when you know the recording was happening) and/or to have ended after a specific time (when you know the recording was happening.

All times are in UTC, which is an hour earlier than Central European Time, 2 hours earlier than Central European Summer Time.


In [7]:
# DATA files from one device, by device ID numbr
devN = 5022299
s_files=df_datafiles.loc[df_datafiles['ID'] == devN,:] 

In [164]:
# DATA files from recordings on a particular day, YYMMDD
recDate = 230201
s_files=df_datafiles.loc[df_datafiles['Date'] == recDate,:]


In [9]:
# DATA files from recordings that lasted longer than N seconds
minDuration = 600  
s_files=df_datafiles.loc[df_datafiles['Duration'] > minDuration,:]


In [187]:
# DATA files from recordings that are smaller than N Bytes
maxfileSize = 200  
s_files=df_datafiles.loc[df_datafiles['FileSize'] < maxfileSize,:]


In [328]:
# recordings started before a certain time 
t = pd.to_datetime('2023-02-01 12:02:00.00+0000')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecStart']< t,:] 

In [146]:
# recordings ended after a certain time
t = pd.to_datetime('2023-01-28 00:00:00.00+0000')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecEnd']> t,:] 

In [188]:
# show subset of files
s_files

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,FullLoc
0,DATA,Bluetooth_Tester_1,3420415,230126,0,DATA-Bluetooth_Tester_1-3420415-23012600.CSV,csv,109,2020-02-02 02:02:00+00:00,C:\Users\Public\Documents\Equivital\Equivital ...
1,DATA,Pilot_31,5022164,230120,0,DATA-Pilot_31-5022164-23012000.CSV,csv,109,2020-02-02 02:02:00+00:00,C:\Users\Public\Documents\Equivital\Equivital ...
2,DATA,Pilot_32,5022060,230123,0,DATA-Pilot_32-5022060-23012300.CSV,csv,109,2020-02-02 02:02:00+00:00,C:\Users\Public\Documents\Equivital\Equivital ...
3,DATA,Pilot_32,5022060,230126,0,DATA-Pilot_32-5022060-23012600.CSV,csv,109,2020-02-02 02:02:00+00:00,C:\Users\Public\Documents\Equivital\Equivital ...
4,DATA,Pilot_32,5022060,230127,2,DATA-Pilot_32-5022060-23012702.CSV,csv,109,2020-02-02 02:02:00+00:00,C:\Users\Public\Documents\Equivital\Equivital ...
...,...,...,...,...,...,...,...,...,...,...
206,DATA,Pilot 57,5022002,230123,0,DATA-Pilot 57-5022002-23012300.CSV,csv,109,2020-02-02 02:02:00+00:00,C:\Users\Public\Documents\Equivital\Equivital ...
207,DATA,Pilot 58,5022240,230120,0,DATA-Pilot 58-5022240-23012000.CSV,csv,109,2020-02-02 02:02:00+00:00,C:\Users\Public\Documents\Equivital\Equivital ...
208,DATA,Pilot 59,5022292,230123,0,DATA-Pilot 59-5022292-23012300.CSV,csv,109,2020-02-02 02:02:00+00:00,C:\Users\Public\Documents\Equivital\Equivital ...
209,DATA,Pilot 44,5022161,230130,1,DATA-Pilot 44-5022161-23013001.CSV,csv,109,2020-02-02 02:02:00+00:00,C:\Users\Public\Documents\Equivital\Equivital ...


These recordings can be checked in View_Equivital_Qiosk_Export.

Now we move them

# Move selected files to suitable project folder

In [148]:
projectsFolder = 'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\' # where qiosk files are moved to after export
print(path) # where the qiosk files are initially saved by the Equivital Qiosk program

C:\Users\Public\Documents\Equivital\Equivital Manager Wizard\


In [149]:
# set project folder name
#projectName = 'AlexanderStandStill'
projectName = 'BatteryTests'
#projectName = 'SoloRecordings'
#projectName = 'Concert230128'
#projectName = 'LauraTestRecording'
projectPath = projectsFolder + projectName + '\\'

### Create folders for project to move files to
Check target location for data files to keep and to remove

In [150]:
# check if the folder exists for project and set up locations
if not os.path.isdir(projectsFolder + projectName):
    os.mkdir(projectsFolder + projectName)
    
if not os.path.isdir(projectsFolder + projectName + '\\' + 'SEM'):
    os.mkdir(projectsFolder + projectName + '\\' + 'SEM')
if not os.path.isdir(projectsFolder + projectName + '\\' + 'CSV'):
    os.mkdir(projectsFolder + projectName + '\\' + 'CSV')

### Move CSV and SEM files to project folder
Move them to project folder and generate their own present recordings file.

In [151]:
for i,row in s_files.iterrows():
    dataFile = row['FullLoc']
    matched = matched_files(dataFile,path) # outputs locations of csv and sem files
    
    for fi in matched:
        fileName = fi.split('\\')[-1]
        devName = row['DevName']
        if fileName.lower().endswith('csv'):
            if not fileName.lower().endswith('Recordings.csv'):
                out_f = projectPath + 'CSV\\' + fileName
                os.rename(fi,out_f)
        if fileName.lower().endswith('sem'):
            if not os.path.isdir(projectsFolder + projectName + '\\SEM\\' + devName):
                os.mkdir(projectsFolder + projectName  + '\\SEM\\' + devName)
            out_f = projectPath + 'SEM\\' + devName  + '\\' + fileName
            os.rename(fi,out_f)
            
dfiles = qiosk_recordings(projectPath,projectName)
dfiles

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,Test3_On,3420755,230120,6,DATA-Test3_On-3420755-23012006.CSV,csv,864823,2023-01-20 15:06:18+00:00,2023-01-21 07:17:23+00:00,...,"[Test3_On, Test5, Test4]",3420755.0,Test5,NaN,0,100,1,1,Stationary,Side
1,DATA,Test5,3420427,230120,4,DATA-Test5-3420427-23012004.CSV,csv,854433,2023-01-20 15:06:20+00:00,2023-01-21 07:07:25+00:00,...,"[Test5, Test3_On, Test4]",3420427.0,Test5,NaN,0,100,1,1,Stationary,Side
2,DATA,Test2,3420426,230120,3,DATA-Test2-3420426-23012003.CSV,csv,1841283,2023-01-20 15:06:22+00:00,2023-01-22 01:39:42+00:00,...,"[Test2, Test1]",3420426.0,Test2,NaN,0,100,1,1,Stationary,Side
3,DATA,Test4,3420456,230120,4,DATA-Test4-3420456-23012004.CSV,csv,820775,2023-01-20 15:06:25+00:00,2023-01-21 06:30:30+00:00,...,"[Test4, Test5, Test3_On]",3420456.0,Test5,NaN,0,100,1,1,Stationary,Side
4,DATA,Pilot_11,5022031,230127,0,DATA-Pilot_11-5022031-23012700.CSV,csv,805075,2023-01-27 11:10:58+00:00,2023-01-29 06:39:13+00:00,...,"[Pilot_11, Pilot_22]",5022031.0,Pilot_11,NaN,0,100,1,1,Stationary,Side
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,DATA,Pilot_31,5022164,230127,101,DATA-Pilot_31-5022164-23012700_0.CSV,csv,815853,2023-01-27 15:40:15+00:00,2023-01-29 11:09:00+00:00,...,"[Pilot_31, Pilot_29, Pilot_12, Pilot_23]",5022164.0,Pilot_23,NaN,16,100,1,1,Stationary,Prone
64,DATA,Pilot_32,5022060,230127,101,DATA-Pilot_32-5022060-23012700_0.CSV,csv,831779,2023-01-27 15:40:15+00:00,2023-01-29 12:31:45+00:00,...,"[Pilot_32, Pilot_1, Pilot_9, Pilot_3, Pilot_4,...",5022060.0,Pilot_9,NaN,16,100,1,1,Stationary,Prone
65,DATA,Pilot_23,5022241,230127,101,DATA-Pilot_23-5022241-23012700_0.CSV,csv,815397,2023-01-27 15:40:15+00:00,2023-01-29 11:09:45+00:00,...,"[Pilot_23, Pilot_25, Pilot_14, Pilot_20]",5022241.0,Pilot_23,NaN,38,98,1,1,Stationary,Prone
66,DATA,Pilot_24,5022110,230127,0,DATA-Pilot_24-5022110-23012700.CSV,csv,831767,2023-01-27 15:40:15+00:00,2023-01-29 12:28:45+00:00,...,"[Pilot_24, Pilot_32, Pilot_1, Pilot_9, Pilot_3]",5022110.0,Pilot_24,NaN,6,100,1,1,Stationary,Prone


# Clear Recordings to ToBeDeleted

In [190]:
# if these are files to be discarded store the DATA file indexes to discardRecs
discardRecs = s_files.index
projTag = 'ToBeDiscarded'
discardPath = 'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\ToBeDeleted\\'

for i,row in s_files.iterrows():
    dataFile = row['FullLoc']
    matched = matched_files(dataFile,path) # outputs locations of csv and sem files
    
    for f in matched:
        fileName = f.split('\\')[-1]
        devName = row['DevName']
        if f.lower().endswith('csv'):
            out_f = discardPath + 'CSV\\' + fileName
            os.rename(f,out_f)
        if f.lower().endswith('sem'):
            if not os.path.isdir(discardPath + 'SEM\\' + devName):
                os.mkdir(discardPath + 'SEM\\' + devName)
            out_f = discardPath + 'SEM\\' + devName + '\\' + fileName
            os.rename(f,out_f)
            
dfiles = qiosk_recordings(discardPath,projTag)
print(len(dfiles))

366


# if Files need to be moved between project folders

Set the origin project name and target project name, collect the existing files at that location, chose the subset of recordings to move or copy, move or copy them, generate recording list in each location

In [52]:
# incase files need to be shifted from a project folder
projectName = 'BatteryTests'
projectPath = projectsFolder + projectName + '\\'
projectPath1 = projectPath
projectName1 = projectName

s_files = qiosk_recordings(projectPath1,projectName1)
s_files 

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,Pilot_11,5022031,230127,0,DATA-Pilot_11-5022031-23012700.CSV,csv,805075,2023-01-27 11:10:58+00:00,2023-01-29 06:39:13+00:00,...,"[Pilot_11, Pilot_22]",5022031,Pilot_11,NaN,0,100,1,1,Stationary,Side


In [ ]:
# get subset of s_files as you see fit

In [ ]:
# project these files should be moved to
projectName = 'Concert230128'
# check if the folder exists for project and set up locations
if not os.path.isdir(projectsFolder + projectName):
    os.mkdir(projectsFolder + projectName)
    
if not os.path.isdir(projectsFolder + projectName + '\\' + 'SEM'):
    os.mkdir(projectsFolder + projectName + '\\' + 'SEM')
if not os.path.isdir(projectsFolder + projectName + '\\' + 'CSV'):
    os.mkdir(projectsFolder + projectName + '\\' + 'CSV')
    
projectPath = projectsFolder + projectName + '\\'
projectPath2 = projectPath
projectName2 = projectName

In [60]:
# move s_files
for i,row in s_files.iterrows():
    dataFile = row['FullLoc']
    matched = matched_files(dataFile,path) # outputs locations of csv and sem files
    
    for fi in matched:
        fileName = fi.split('\\')[-1]
        devName = row['DevName']
        if fileName.lower().endswith('csv'):
            if not fileName.lower().endswith('Recordings.csv'):
                out_f = projectPath + 'CSV\\' + fileName
                os.rename(fi,out_f)
        if fileName.lower().endswith('sem'):
            if not os.path.isdir(projectsFolder + projectName + '\\SEM\\' + devName):
                os.mkdir(projectsFolder + projectName  + '\\SEM\\' + devName)
            out_f = projectPath + 'SEM\\' + devName  + '\\' + fileName
            os.rename(fi,out_f)

['C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\LauraTestRecording\\LauraTestRecording_QIOSK_Recordings.csv', 'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\LauraTestRecording\\CSV\\DATA-Pilot_11-5022031-23012700.CSV']
C:\Users\fourMs lab\Documents\Equivital\ProjectData\LauraTestRecording\CSV\DATA-Pilot_11-5022031-23012700.CSV
C:\Users\fourMs lab\Documents\Equivital\ProjectData\LauraTestRecording\SEM\Pilot_11\23012700.SEM


In [60]:
# COPY s_files
for i,row in s_files.iterrows():
    dataFile = row['FullLoc']
    matched = matched_files(dataFile,path) # outputs locations of csv and sem files
    
    for fi in matched:
        fileName = fi.split('\\')[-1]
        devName = row['DevName']
        if fileName.lower().endswith('csv'):
            if not fileName.lower().endswith('Recordings.csv'):
                out_f = projectPath + 'CSV\\' + fileName
                os.system('cp ' + fi + ' ' + out_f)
        if fileName.lower().endswith('sem'):
            if not os.path.isdir(projectsFolder + projectName + '\\SEM\\' + devName):
                os.mkdir(projectsFolder + projectName  + '\\SEM\\' + devName)
            out_f = projectPath + 'SEM\\' + devName  + '\\' + fileName
            os.system('cp ' + fi + ' ' + out_f)


['C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\LauraTestRecording\\LauraTestRecording_QIOSK_Recordings.csv', 'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\LauraTestRecording\\CSV\\DATA-Pilot_11-5022031-23012700.CSV']
C:\Users\fourMs lab\Documents\Equivital\ProjectData\LauraTestRecording\CSV\DATA-Pilot_11-5022031-23012700.CSV
C:\Users\fourMs lab\Documents\Equivital\ProjectData\LauraTestRecording\SEM\Pilot_11\23012700.SEM


In [ ]:
qiosk_recordings(projectPath1,projectName1)
dfiles = qiosk_recordings(projectPath2,projectName2)